In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from matplotlib import pyplot as plt

In [2]:
# Modify the PINN class to accept a different number of nodes in each the hidden layers
class PINN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, act=nn.ReLU()):
        super(PINN, self).__init__()
        
        self.layers = nn.ModuleList()
        layer_sizes = [input_size] + hidden_sizes + [output_size]

        for i in range(len(layer_sizes) - 1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))

        self.activation = act

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = self.activation(layer(x))
        output = self.layers[-1](x)
        return output

In [3]:
def random_domain_points(n):
    x = torch.rand(n, 2, requires_grad=True)
    return x

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('device:', device)
else:
    device = torch.device("cpu")
    print('CUDA is not available. Using CPU.')

CUDA is not available. Using CPU.


In [6]:
# Print the PINN architecture
pinn = PINN(2, [10, 50, 50, 50, 10], 1, act=torch.nn.Sigmoid()).to(device)

learning_rate = 0.01
optimizer = optim.Adam(pinn.parameters(), lr=learning_rate)

epochs = int(1e3)
convergence_data = torch.empty((epochs), device=device)

gamma1 = 100.
loss_list = []

In [7]:
x = random_domain_points(5).to(device)
u = pinn(x)

In [13]:
linear_mod = nn.Sequential(
                nn.Linear(2,1))

In [21]:
class Model(torch.nn.Module):
    def __init__(self) -> None:
        super(Model,self).__init__()
        self.layer01 = torch.nn.Linear(2,1)
    
    def forward(self,x):
        inputs = x
        output = torch.pow(self.layer01(inputs),2)
        return output

$m(x,y) = (ax + by + c)^2$

$m_{xx}(x,y) = 2a^2$

In [32]:
model = Model()
model.state_dict()

OrderedDict([('layer01.weight', tensor([[0.1577, 0.1394]])),
             ('layer01.bias', tensor([-0.1154]))])

In [45]:
x = torch.tensor([[1.0, 0.1]], requires_grad=True)
u = model(x)

$m_x(x,y) = 2a(ax + by + c)$

In [46]:
# Compute derivatives
u_x = torch.autograd.grad(outputs=u, inputs=x, create_graph=True,
                          grad_outputs=torch.ones_like(u))[0]

In [47]:
2*0.1577*(0.1577 + 0.1394 -0.1154)

0.057308180000000014

In [49]:
2*0.1394*(0.1577 + 0.1394*0.1 -0.1154)

0.015679712000000002

In [48]:
u_x

tensor([[0.0178, 0.0157]], grad_fn=<MmBackward0>)

In [52]:
u_xx = torch.autograd.grad(outputs=u_x, inputs=x, create_graph=True,
                                   grad_outputs=torch.ones_like(u_x))[0]

In [53]:
u_xx

tensor([[0.0937, 0.0829]], grad_fn=<MmBackward0>)

In [38]:
2*(0.1577)**2

0.049738580000000004